In [9]:
import sys
sys.path.insert(0, 'src')
from read_genome import *
from bm_preproc import *


In [5]:

!wget --no-check -O resources/excerpt.fasta http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2024-10-24 10:43:07--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.32.84.71, 13.32.84.169, 13.32.84.231, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.32.84.71|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘resources/excerpt.fasta’

resources/excerpt.f 100%[===================>] 791.12K  1.20MB/s    in 0.6s    

2024-10-24 10:43:08 (1.20 MB/s) - ‘resources/excerpt.fasta’ saved [810105/810105]



In [17]:
human_chromosome = readFasta('resources/excerpt.fasta')

### Naive

In [5]:
#count th numner of character comparisons and the number of alignments
def naive_with_counts(p, t):
    occurrences = []
    no_comp = 0
    no_alignments = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        no_alignments += 1
        for j in range(len(p)):  # loop over characters
            no_comp += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences, no_alignments, no_comp

Example 1

In [6]:
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


Example 2

In [7]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


### Booyer-Moore

In [24]:
def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    no_comp = 0
    no_aligments = 0
    while i < len(t) - len(p) + 1:
        no_aligments+=1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            no_comp += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, no_aligments, no_comp

Example 1

In [25]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


Example 2

In [26]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


Q1. How many alignments does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1? (Don't consider reverse complements.)

In [100]:
s = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
occurrences, num_alignments, num_character_comparisons= naive_with_counts(s, human_chromosome)
num_alignments

799954

Q2. How many character comparisons does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)


In [99]:
num_character_comparisons

984143

Q3. How many alignments does Boyer-Moore try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1? (Don't consider reverse complements.)

In [101]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
alphabet = 'ACGT'
p_bm = BoyerMoore(p, alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, human_chromosome)
num_alignments

127974

Q4. Index-assisted approximate matching. Implement the pigeonhole principle using Index to find exact matches for the partitions. Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). We will use an 8-mer index.

Write a function that, given a length-24 pattern P and given an 
Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

In [40]:
from kmer_index import *
def approximate_match(p, t, n):
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    index = Index(t, 8)
    hits = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = index.query(p[start:end])
        hits+=len(matches)
        # p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        # matches = boyer_moore(p[start:end], p_bm, t)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), hits

How many times does the string 
GGCGCGGTGGCTCACGCCTGTAATGGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.

Hint 2: You can check your work by comparing the output of your new function to that of the naive_2mm function implemented in the previous module

In [43]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
occ, hits = approximate_match(p, human_chromosome,2)
len(occ)

19

Check with naive_2mm

In [42]:
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        mismatches = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mismatches += 1
                if mismatches > 2:
                    match = False
                    break

        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences
len(naive_2mm(p, human_chromosome))

19

Q5. Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of 
GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?
(Don't consider reverse complements.)

Hint: You should be able to use the boyer_moore function (or the slower 
naive function) to double-check your answer.

In [44]:
hits

90

Q6. Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, as we discussed in the "Variations on k-mer indexes" video.  We'll consider subsequences involving every N characters.  For example, if we split 
ATATAT into two substring partitions, we would get partitions 
ATA (the first half) and 
TAT (second half).  But if we split 
ATATAT into two  subsequences  by taking every other character, we would get 
AAA (first, third and fifth characters) and 
TTT (second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition.  Splitting a length-6 pattern into two substrings could be represented as 
111222, and splitting into two subsequences of every other character could be represented as 
121212

In [176]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits
    
    def query_subseq(self, p):
        n = 2 #number of maximum mismatches
        all_matches = set()
        hits = 0
        for i in range(n+1):
            matches = self.query(p[i:])
            hits+=len(matches)
            mismatches = 0
            #check the matches found
            for m in matches:
                if m < i or m-i+len(p) > len(t):
                    continue
                for j in range(0, len(p)):
                    if not p[j] == t[m - i + j]:
                        mismatches += 1
                        if mismatches > n:
                            break
                if mismatches <= n:
                    all_matches.add(m - i)         
        return list(all_matches), hits

In [177]:
t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)
# subseq_ind.index

In [178]:
occ, hits = subseq_ind.query_subseq(p)
occ, hits

([0, 14], 6)

In [137]:
!wget -O resources/king_john http://www.gutenberg.org/ebooks/1110.txt.utf-8

--2024-10-24 13:31:01--  http://www.gutenberg.org/ebooks/1110.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/ebooks/1110.txt.utf-8 [following]
--2024-10-24 13:31:02--  https://www.gutenberg.org/ebooks/1110.txt.utf-8
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://www.gutenberg.org/cache/epub/1110/pg1110.txt [following]
--2024-10-24 13:31:02--  http://www.gutenberg.org/cache/epub/1110/pg1110.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/1110/pg1110.txt [following]
--2024-10-24 13:31:03--  https://www.gutenberg.org/cache/e

In [179]:
t = open('resources/king_john').read()
p = 'English measure backward'
subseq_ind = SubseqIndex(t, 8, 3)
occurrences, num_index_hits = subseq_ind.query_subseq(p)
occurrences, num_index_hits

([110323], 3)

Write a function that, given a length-24 pattern P and given a 
SubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

When using this function, how many total index hits are there when searching for 
GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)

In [181]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
subseq_ind = SubseqIndex(human_chromosome, 8, 3)
occ, hits = subseq_ind.query_subseq(p)
occ, hits

([], 79)